In [1]:
import tensorflow as tf
from tensorflow.keras import preprocessing

import numpy as np

In [2]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

targets =[[1], [0], [1], [1], [0], [1]]

In [26]:
tokenizer = preprocessing.text.Tokenizer()
# fit_on_texts()는 samples 리스트에서 문장을 읽어서
# word_index를 만들어 준다
tokenizer.fit_on_texts(samples)

# 만들어진 word_index를 확인
word_index = tokenizer.word_index

# samples의 문장을 word_index를 활용하여 문자를 숫자로 변환
# '너 오늘 이뻐 보인다' -> [4, 1, 5, 6]
sequences = tokenizer.texts_to_sequences(samples)
# 입력된 문장과 레이블을 np.array타입으로 변환
input_sequences = np.array(sequences)
labels = np.array(targets)


In [27]:
print('수치화된 텍스트 데이터 : \n',sequences)
print('sequences type : {}'.format(type(sequences)))
print('\n단어 인덱스 :\n',word_index)
print('\n라벨 : ',labels)

수치화된 텍스트 데이터 : 
 [[4, 1, 5, 6], [7, 1, 8, 9], [10, 2, 3, 11], [12, 2, 3, 13], [14, 1, 15, 16], [17, 18, 19, 20]]
sequences type : <class 'list'>

단어 인덱스 :
 {'오늘': 1, '좋은': 2, '일이': 3, '너': 4, '이뻐': 5, '보인다': 6, '나는': 7, '기분이': 8, '더러워': 9, '끝내주는데': 10, '있나봐': 11, '나': 12, '생겼어': 13, '아': 14, '진짜': 15, '짜증나': 16, '환상적인데': 17, '정말': 18, '좋은거': 19, '같아': 20}

라벨 :  [[1]
 [0]
 [1]
 [1]
 [0]
 [1]]


In [4]:
from tensorflow.keras import backend
from tensorflow.keras import layers

In [5]:
batch_size = 2
num_epochs = 100

In [6]:
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

# Keras Sequential

In [7]:
model = tf.keras.Sequential([
# 들어오는 데이터가 원핫벡터로 들어오면 원핫인코딩으로 바꿔준다
# [4, 1, 5, 6] -> [00001000....00],[01000....00],[00000100....00],[000000100....00]
layers.Embedding(vocab_size, emb_size, input_length = 4),
# reduce_mean : 평균값구하기
layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
layers.Dense(hidden_dimension, activation='relu'),
layers.Dense(output_dimension, activation='sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 4, 128)            2688      
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 645us/step - loss: 0.6933 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 459us/step - loss: 0.6721 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 474us/step - loss: 0.6549 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 533us/step - loss: 0.6375 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 459us/step - loss: 0.6179 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 422us/step - loss: 0.5960 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 477us/step - loss: 0.5708 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 439us/step - loss: 0.5408 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 436us/step - loss: 0.5071 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 446us/step - loss: 0.4681 - accuracy: 1.0000
Epoch 11/

# Keras Functional API

In [10]:
# Keras Functional API는 입력할 레이어, 출력할 레이어를 지정할 수 있다
inputs = layers.Input(shape = (4, ))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis=1)
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)

In [11]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 4, 128)            2688      
_________________________________________________________________
tf_op_layer_Mean (TensorFlow [(None, 128)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 526us/step - loss: 0.6933 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 540us/step - loss: 0.6734 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 417us/step - loss: 0.6565 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 448us/step - loss: 0.6393 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 477us/step - loss: 0.6211 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 445us/step - loss: 0.6004 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 539us/step - loss: 0.5728 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 465us/step - loss: 0.5426 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 432us/step - loss: 0.5066 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 441us/step - loss: 0.4647 - accuracy: 1.0000
Epoch 11/

# Keras Custom Model

In [14]:
class CustomModel(tf.keras.Model):

    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)
        x = self.output_layer(x)
        
        return x

In [15]:
model = CustomModel(vocab_size = vocab_size,
            embed_dimension=emb_size,
            hidden_dimension=hidden_dimension,
            output_dimension=output_dimension)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 510us/step - loss: 0.6915 - accuracy: 0.6667
Epoch 2/100
3/3 [==============================] - 0s 525us/step - loss: 0.6718 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 482us/step - loss: 0.6557 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 465us/step - loss: 0.6394 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 445us/step - loss: 0.6210 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 455us/step - loss: 0.5977 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 465us/step - loss: 0.5707 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 480us/step - loss: 0.5415 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 451us/step - loss: 0.5052 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 453us/step - loss: 0.4647 - accuracy: 1.0000
Epoch 11/

# Keras Custom Layer

In [16]:
class CustomLayer(layers.Layer):

    def __init__(self, hidden_dimension, output_dimension, **kwargs):
        self.hidden_dimension = hidden_dimension
        self.output_dimension = output_dimension
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.dense_layer1 = layers.Dense(self.hidden_dimension, activation = 'relu')
        self.dense_layer2 = layers.Dense(self.output_dimension)

    def call(self, inputs):
        hidden_output = self.dense_layer1(inputs)
        return self.dense_layer2(hidden_output)

    # Optional
    def get_config(self):
        base_config = super(CustomLayer, self).get_config()
        base_config['hidden_dim'] = self.hidden_dimension
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [17]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, emb_size, input_length = 4),
    layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
    CustomLayer(hidden_dimension, output_dimension),
    layers.Activation('sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 547us/step - loss: 0.6869 - accuracy: 0.6667
Epoch 2/100
3/3 [==============================] - 0s 888us/step - loss: 0.6675 - accuracy: 0.6667
Epoch 3/100
3/3 [==============================] - 0s 536us/step - loss: 0.6506 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 527us/step - loss: 0.6330 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 608us/step - loss: 0.6141 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 425us/step - loss: 0.5898 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 483us/step - loss: 0.5646 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 445us/step - loss: 0.5283 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 461us/step - loss: 0.4943 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 424us/step - loss: 0.4496 - accuracy: 1.0000
Epoch 11/